<a href="https://colab.research.google.com/github/Hamburger4455/HME-recognition/blob/main/the_project_10423158.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
# Run this to start all:
# !pip install torchmetrics pytorch_lighting

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchmetrics.text import EditDistance

import numpy as np
import pandas as pd
import os

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
import wandb

import xml.etree.ElementTree as ET
from torch.nn.utils.rnn import pad_sequence

import matplotlib.pyplot as plt
from helper import *
from collections.abc import Sequence
from typing import Literal, Optional, Union

import json

In [2]:
from torchaudio.models.decoder import cuda_ctc_decoder

In [11]:
# Task 1

In [3]:
paths = [
    "dataset/crohme2019_train.txt",
    "dataset/crohme2019_test.txt",
    "dataset/crohme2019_valid.txt",
]
df = pd.read_csv(paths[1], sep="\t", header=None, names=["path", "label"]).dropna().astype(str)
df.head()

,path,label
0,crohme2019/test/UN19_1032_em_455.inkml,4 Right n Right - Right 4 Right ) NoRel ( NoRe...
1,crohme2019/test/UN19_1044_em_632.inkml,\phi Sub 0 NoRel = Right d Right x Sup 1 Right...
2,crohme2019/test/UN19wb_1111_em_1039.inkml,9 Right \times Right 9 Right + Right 1 Right 3...
3,crohme2019/test/UN19_1028_em_394.inkml,\tan Right \alpha Right \tan Right \theta Sup ...
4,crohme2019/test/UN19_1005_em_64.inkml,a Right = NoRel 1 NoRel - Below \sqrt Inside 2...


In [4]:
def get_unique_chars(paths) -> set:
    res = set()
    for i in df['label'].apply(lambda x: x.strip().split()):
        res.update(i)
    return res

len(get_unique_chars(paths[0]))

108

In [5]:
unique_chars = get_unique_chars(paths[0])
unique_chars

{'!',
 '(',
 ')',
 '+',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '=',
 'A',
 'Above',
 'B',
 'Below',
 'C',
 'COMMA',
 'E',
 'F',
 'G',
 'H',
 'I',
 'Inside',
 'L',
 'M',
 'N',
 'NoRel',
 'P',
 'R',
 'Right',
 'S',
 'Sub',
 'Sup',
 'T',
 'V',
 'X',
 'Y',
 '[',
 '\\Delta',
 '\\alpha',
 '\\beta',
 '\\cos',
 '\\div',
 '\\exists',
 '\\forall',
 '\\gamma',
 '\\geq',
 '\\gt',
 '\\in',
 '\\infty',
 '\\int',
 '\\lambda',
 '\\ldots',
 '\\leq',
 '\\lim',
 '\\log',
 '\\lt',
 '\\mu',
 '\\neq',
 '\\phi',
 '\\pi',
 '\\pm',
 '\\prime',
 '\\rightarrow',
 '\\sigma',
 '\\sin',
 '\\sqrt',
 '\\sum',
 '\\tan',
 '\\theta',
 '\\times',
 '\\{',
 '\\}',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|'}

In [6]:
def chr2idx(unique_chars: set) -> dict:
    unique_chars.add('')
    return {char: idx for idx, char in enumerate(sorted(unique_chars))}

vocab = chr2idx(get_unique_chars(paths[0]))

assert vocab[""] == 0
assert vocab["|"] == 108
assert vocab["\\pi"] == 68
assert vocab["\\exists"] == 51
assert len(vocab) == 109

In [7]:
class Vocab:
    """
    Attributes:
        paths (list): A list of file paths to the annotation files.
        char2idx (dict): A dictionary mapping characters to their corresponding indices.  This is the primary vocabulary.
        idx2char (dict): A reverse mapping from indices to characters, facilitating decoding.

    Methods:
        build_vocab(): Builds the vocabulary from the specified annotation files, sorting characters lexicographically and adding a blank character.
        get_vocab(): Returns the character-to-index vocabulary.
        save_vocab(path): Saves the vocabulary to a JSON file.
        encode(tokens): Converts a list of tokens (characters) into a list of their corresponding indices.
        decode(ids): Converts a list of indices back into a list of tokens (characters).
    """
    def __init__(self, vocab_file=None):
        self.char2idx = {}
        if vocab_file is not None:
            self.char2idx = json.load(open(vocab_file))
        self.idx2char = {v: k for k, v in self.char2idx.items()}

    def build_vocab(self, annotations) -> None:
        vocab_set = set()
        for path in annotations:
            df = pd.read_csv(path, sep="\t", header=None, names=["path", "label"]).dropna().astype(str)
            for label in df["label"]:
                tokens = label.strip().split()
                vocab_set.update(tokens)
        vocab_set.add("")
        vocab = {char: idx for idx, char in enumerate(sorted(vocab_set))}
        self.char2idx = vocab
        self.idx2char = {idx: char for char, idx in vocab.items()}

    def get_vocab(self) -> dict:
        return self.char2idx

    def save_vocab(self, path: str) -> None:
        with open(path, 'w') as f:
            json.dump(self.char2idx, f)

    def encode(self, tokens):
        return [self.char2idx[token] for token in tokens]

    def decode(self, ids):
        return [self.idx2char[id] for id in ids]


In [8]:
annotations = [
    "dataset/crohme2019_train.txt",
    "dataset/crohme2019_test.txt",
    "dataset/crohme2019_valid.txt",
]

vocab = Vocab()
vocab.build_vocab(annotations)
vocab.save_vocab('vocab.json')


In [9]:
# Step 9: Test
vocab = Vocab('vocab.json')
input = '- Right \\sqrt Inside 2'.split()

assert vocab.encode(input) == [5, 37, 74, 30, 10]
assert vocab.decode(vocab.encode(input)) == input


In [19]:
# Task 2

In [10]:
class Segment(object):
    """Class to reprsent a Segment compound of strokes (id) with an id and label."""

    __slots__ = ("id", "label", "strId")

    def __init__(self, *args):
        if len(args) == 3:
            self.id = args[0]
            self.label = args[1]
            self.strId = args[2]
        else:
            self.id = "none"
            self.label = ""
            self.strId = set([])


class Inkml(object):
    """Class to represent an INKML file with strokes, segmentation and labels"""

    __slots__ = ("fileName", "strokes", "strkOrder", "segments", "truth", "UI")

    NS = {
        "ns": "http://www.w3.org/2003/InkML",
        "xml": "http://www.w3.org/XML/1998/namespace",
    }

    def __init__(self, *args):
        self.fileName = None
        self.strokes = {}
        self.strkOrder = []
        self.segments = {}
        self.truth = ""
        self.UI = ""
        if len(args) == 1:
            self.fileName = args[0]
            self.loadFromFile()

    def fixNS(self, ns, att):
        """Build the right tag or element name with namespace"""
        return "{" + Inkml.NS[ns] + "}" + att

    def loadFromFile(self):
        """load the ink from an inkml file (strokes, segments, labels)"""
        tree = ET.parse(self.fileName)
        # # ET.register_namespace();
        root = tree.getroot()
        for info in root.findall("ns:annotation", namespaces=Inkml.NS):
            if "type" in info.attrib:
                if info.attrib["type"] == "truth":
                    self.truth = info.text.strip()
                if info.attrib["type"] == "UI":
                    self.UI = info.text.strip()
        for strk in root.findall("ns:trace", namespaces=Inkml.NS):
            self.strokes[strk.attrib["id"]] = strk.text.strip()
            self.strkOrder.append(strk.attrib["id"])
        segments = root.find("ns:traceGroup", namespaces=Inkml.NS)
        if segments is None or len(segments) == 0:
            return
        for seg in segments.iterfind("ns:traceGroup", namespaces=Inkml.NS):
            id = seg.attrib[self.fixNS("xml", "id")]
            label = seg.find("ns:annotation", namespaces=Inkml.NS).text
            strkList = set([])
            for t in seg.findall("ns:traceView", namespaces=Inkml.NS):
                strkList.add(t.attrib["traceDataRef"])
            self.segments[id] = Segment(id, label, strkList)

    def getTraces(self, height=256):
        traces_array = [
            np.array(
                [p.strip().split() for p in self.strokes[id].split(",")], dtype="float"
            )
            for id in self.strkOrder
        ]

        ratio = height / (
            (
                np.concatenate(traces_array, 0).max(0)
                - np.concatenate(traces_array, 0).min(0)
            )[1]
            + 1e-6
        )
        return [(trace * ratio).astype(int).tolist() for trace in traces_array]

    def view(self):
        plt.figure(figsize = (16, 4))
        plt.axis("off")
        for trace in self.getTraces():
            trace_arr = np.array(trace)
            plt.plot(trace_arr[:, 0], -trace_arr[:, 1])  # invert y coordinate

In [11]:
def feature_extraction(traces):
    """
    Convert a list of strokes (each a list of points) into a
    (T, 4) feature matrix where each row is
        (Δx / d ,  Δy / d ,  d ,  pen_up)
    """

    # ---- 1. flatten strokes & keep only x,y ----
    traces_xy = [np.asarray(t, dtype=float)[:, :2] for t in traces]   # drop t coord if present
    lens = [len(t) for t in traces_xy]                                # length per stroke
    cum_lens = np.cumsum(lens)                                        # to mark stroke boundaries
    pts = np.concatenate(traces_xy, axis=0)                           # (N,2)

    # ---- 2. first-order deltas ----
    deltas = pts[1:] - pts[:-1]                                       # (N-1,2)

    # ---- 3. Euclidean distance per step ----
    dist = np.linalg.norm(deltas, axis=1)                             # (N-1,)

    # ---- 4. pen-up indicator (1 where next point starts a new stroke) ----
    pen_up = np.zeros_like(dist, dtype=float)
    pen_up[cum_lens[:-1] - 1] = 1.0                                   # mark end-of-stroke

    # ---- 5. remove zero-length steps (d == 0) ----
    keep = dist > 0
    deltas, dist, pen_up = deltas[keep], dist[keep], pen_up[keep]

    # ---- 6. normalise Δx, Δy by distance ----
    deltas_norm = deltas / dist[:, None]                              # broadcasting

    # ---- 7. assemble final feature matrix ----
    feature = np.hstack([deltas_norm, dist[:, None], pen_up[:, None]])  # (T,4)
    return feature


In [12]:
class InkmlDataset(Dataset):
    def __init__(self, annotation, root_dir, vocab):
        """
        Args:
            annotation (str) : path to *.txt annotation file  (path <tab> label ...)
            root_dir  (str) : dataset root (prepended to each relative path)
            vocab     (Vocab) : vocabulary object (implements .encode())
        """
        self.vocab = vocab
        df = pd.read_csv(annotation, sep="\t", header=None, names=["path", "label"]).dropna()

        # store full inkml paths and tokenised labels
        self.inks   = [os.path.join(root_dir, p) for p in df["path"].astype(str)]
        self.labels = [lbl.strip().split()       for lbl in df["label"].astype(str)]

    def __len__(self):
        return len(self.inks)

    def __getitem__(self, idx):
        # ----- 1. load inkml & extract features -----
        ink  = Inkml(self.inks[idx])
        feat = feature_extraction(ink.getTraces())        # (T,4)  numpy

        # ----- 2. encode label with vocab -----
        target = self.vocab.encode(self.labels[idx])      # list[int]

        # ----- 3. convert to tensors -----
        input_tensor  = torch.tensor(feat,    dtype=torch.float32)   # (T,4)
        target_tensor = torch.tensor(target, dtype=torch.long)       # (U,)

        input_len  = torch.tensor([input_tensor.size(0)], dtype=torch.long)
        label_len  = torch.tensor([target_tensor.size(0)], dtype=torch.long)

        return input_tensor, target_tensor, input_len, label_len


In [13]:
# example — change paths as needed
ann  = "dataset/crohme2019_train.txt"
root = "dataset"                 # or "dataset/crohme2019" if paths are deeper
ds   = InkmlDataset(ann, root, vocab)

sample = ds[0]                   # should return 4 tensors
print([t.shape for t in sample[:2]], sample[2], sample[3])


[torch.Size([363, 4]), torch.Size([21])] tensor([363]) tensor([21])


In [24]:
# task 3

In [25]:
# Implement collate_fn

In [14]:
def collate_fn(batch):
    """Create batch by padding variable-length input and label sequences"""
    inputs, targets, input_lens, label_lens = zip(*batch)

    # Pad inputs (each is (Ti, 4)) => (maxT, B, 4)
    padded_inputs = pad_sequence(inputs, batch_first=True)  # (B, T_max, 4)

    # Pad labels (each is (Li,)) => (B, L_max)
    padded_labels = pad_sequence(targets, batch_first=True, padding_value=-1)  # use -1 to avoid CTC blank confusion

    input_lens = torch.stack(input_lens)
    label_lens = torch.stack(label_lens)

    return padded_inputs, padded_labels, input_lens, label_lens


In [27]:
# test

In [15]:
dataset = InkmlDataset(
    annotation="dataset/crohme2019_train.txt",
    root_dir="dataset",
    vocab=vocab
)

features, labels, input_lens, label_lens = collate_fn([dataset[0], dataset[1]])

import numpy.testing as npt
assert type(input_lens) == torch.Tensor
assert type(label_lens) == torch.Tensor


In [29]:
# Implement InkmlDataset_PL LightningDataModule

In [16]:
class InkmlDataset_PL(pl.LightningDataModule):
    """
    PyTorch Lightning data module for handling the INKML dataset.
    """

    def __init__(
        self,
        batch_size: int = 10,
        workers: int = 5,
        train_data: str = "",
        val_data: str = "",
        test_data: str = "",
        root_dir: str = "dataset",
        vocab=None,
    ):
        super().__init__()
        self.batch_size = batch_size
        self.workers = workers
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        self.root_dir = root_dir
        self.vocab = vocab

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            self.train_dataset = InkmlDataset(self.train_data, root_dir=self.root_dir, vocab=self.vocab)
            self.val_dataset = InkmlDataset(self.val_data, root_dir=self.root_dir, vocab=self.vocab)
        if stage == "test" or stage is None:
            self.test_dataset = InkmlDataset(self.test_data, root_dir=self.root_dir, vocab=self.vocab)

    def custom_collate_fn(self, data):
        return collate_fn(data)

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=self.workers,
            collate_fn=self.custom_collate_fn,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=self.workers,
            collate_fn=self.custom_collate_fn,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=self.workers,
            collate_fn=self.custom_collate_fn,
        )


In [31]:
# task 4

In [17]:
class LSTM_TemporalClassification(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM_TemporalClassification, self).__init__()

        # 1. Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )

        # 2. Fully connected layer
        self.fc = nn.Linear(hidden_size * 2, num_classes)

        # 3. LogSoftmax for CTC Loss
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        lstm_out, _ = self.lstm(x)  # (batch_size, seq_len, hidden_size*2)
        logits = self.fc(lstm_out)  # (batch_size, seq_len, num_classes)
        output = self.softmax(logits)
        return output


In [33]:
# test

In [18]:
model = LSTM_TemporalClassification(4, 256, 2, 109)
dummy = torch.rand((10, 100, 4))  # batch_size=10, seq_len=100, input_size=4
assert model.forward(dummy).shape == (10, 100, 109)
print("✅ Model passes shape test")

✅ Model passes shape test


In [35]:
# Task 6

In [19]:
class MathOnlineModel(pl.LightningModule):
    def __init__(
        self,
        lr=0.001,
        input_size=4,
        output_size=109,
        hidden_size=256,
        num_layers=2,
        decoder=None,
    ):
        super().__init__()
        self.model = LSTM_TemporalClassification(
            input_size, hidden_size, num_layers, output_size
        )
        self.criterion = nn.CTCLoss(blank=0, zero_infinity=True)
        self.lr = lr
        self.decoder = decoder

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y, x_lens, y_lens = batch

        log_probs = self.forward(x)                  # (N, T, C)
        log_probs = log_probs.permute(1, 0, 2)       # → (T, N, C) for CTC loss

        loss = self.criterion(log_probs, y, x_lens, y_lens)

        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y, x_lens, y_lens = batch

        log_probs = self.forward(x)
        log_probs = log_probs.permute(1, 0, 2)

        loss = self.criterion(log_probs, y, x_lens, y_lens)

        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y, x_lens, y_lens = batch

        log_probs = self.forward(x)
        log_probs = log_probs.permute(1, 0, 2)

        loss = self.criterion(log_probs, y, x_lens, y_lens)

        self.log("test_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)


In [37]:
# Task 7

In [20]:
# --- Setup WandB ---
student_id = "10423158"  # ✅ your student ID
api_key = os.environ.get("WANDB_API_KEY", "acc59418885db7ca24d7bf62c50c9fa852416364")  # ✅ your WandB key

if api_key == "":
    raise ValueError("Please set your wandb key in the code or in the environment variable WANDB_API_KEY")
else:
    print("WandB API key is set. Proceeding with login...")

wandb.login(key=api_key)


WandB API key is set. Proceeding with login...


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 10423158 (10423158-vietnamese-german-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [21]:
# --- WandB Logger ---
wandb_logger = WandbLogger(
    entity="cuong-nt-vgu-ai-2025",  # DO NOT CHANGE THIS
    project="math_online_2025",    # DO NOT CHANGE THIS
    name=f"{student_id}_run_1",
    config={
        "student_id": student_id,  # DO NOT CHANGE THIS
        "model": "LSTM_TemporalClassification",
        # your hyperparameters
    },
    log_model=True,
    save_dir="wandb_logs",
)


In [22]:
# --- Trainer ---
trainer = Trainer(
    callbacks = [
        LearningRateMonitor(logging_interval='step'),
        ModelCheckpoint(filename='{epoch}-{val_loss:.4f}', save_top_k=5, monitor='val_loss', mode='min'),
    ],
    logger = wandb_logger,
    check_val_every_n_epoch=1,
    fast_dev_run=False,  # enable for testing model
    default_root_dir='checkpoint',
    deterministic=False,
    max_epochs=20,
    log_every_n_steps=50,
    devices = "auto",
)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [36]:
# --- Model and DataModule ---
model = MathOnlineModel()
# model = MathOnlineModel.load_from_checkpoint('path/to/your_checkpoint.ckpt')

vocab = Vocab('vocab.json')  # ✅ Load the saved vocabulary

dm = InkmlDataset_PL(
    root_dir='dataset',
    train_data='dataset/crohme2019_train.txt',
    val_data='dataset/crohme2019_valid.txt',
    test_data='dataset/crohme2019_test.txt',
    vocab=vocab,                    # ✅ REQUIRED!
    batch_size=32,
    workers=2
)


In [24]:
# --- Start Training ---
trainer.fit(model, dm)


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                        | Params | Mode 
------------------------------------------------------------------
0 | model     | LSTM_TemporalClassification | 2.2 M  | train
1 | criterion | CTCLoss                     | 0      | train
------------------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.678     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
# Task 8

In [32]:
for root, dirs, files in os.walk(".", topdown=True):
    for name in files:
        if name.endswith(".ckpt"):
            print(os.path.join(root, name))

./wandb_logs/math_online_2025/ix3ddrcw/checkpoints/epoch=15-val_loss=0.7205.ckpt
./wandb_logs/math_online_2025/ix3ddrcw/checkpoints/epoch=19-val_loss=0.6619.ckpt
./wandb_logs/math_online_2025/ix3ddrcw/checkpoints/epoch=17-val_loss=0.6740.ckpt
./wandb_logs/math_online_2025/ix3ddrcw/checkpoints/epoch=16-val_loss=0.6889.ckpt
./wandb_logs/math_online_2025/ix3ddrcw/checkpoints/epoch=18-val_loss=0.6744.ckpt


In [38]:
# --- Load Saved Vocabulary ---
vocab = Vocab("vocab.json")  # ✅ REQUIRED!

# --- Load Data Module ---
dm = InkmlDataset_PL(
    root_dir="dataset",
    train_data="dataset/crohme2019_train.txt",
    val_data="dataset/crohme2019_valid.txt",
    test_data="dataset/crohme2019_test.txt",
    vocab=vocab,
    batch_size=32,
    workers=2
)

# --- Load Best Checkpoint ---
model = MathOnlineModel.load_from_checkpoint(
    "wandb_logs/math_online_2025/ix3ddrcw/checkpoints/epoch=19-val_loss=0.6619.ckpt"
)

# --- Run Testing ---
trainer = Trainer(devices=1)
trainer.test(model, datamodule=dm)

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.6298266053199768     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6298266053199768}]

In [ ]:
# Task 9

In [64]:
class GreedyCTCDecoder(nn.Module):
    def __init__(self, vocab):
        super().__init__()
        self.vocab = vocab
        self.blank = self.vocab.char2idx[""]  # ✅ use correct key

    def forward(self, emission: torch.Tensor) -> list:
        """Greedy decode CTC emissions."""
        indices = torch.argmax(emission, dim=-1).tolist()

        output_seq = []
        prev_idx = self.blank
        for idx in indices:
            if idx != self.blank and idx != prev_idx:
                output_seq.append(self.vocab.idx2char[idx])
            prev_idx = idx

        return output_seq


In [65]:
# Load model
# Load model (✅ FIXED checkpoint path)
model = MathOnlineModel.load_from_checkpoint('wandb_logs/math_online_2025/ix3ddrcw/checkpoints/epoch=19-val_loss=0.6619.ckpt')
model.eval()


# Load vocab
with open('vocab.json') as f:
    vocab = Vocab('vocab.json')

# Load one sample from dataset
dataset = InkmlDataset('dataset/crohme2019_test.txt', 'dataset', vocab)
feature, label, input_len, label_len = dataset[0]

# Run model to get emissions
with torch.no_grad():
    emissions = model(feature.unsqueeze(0).to(model.device))
    emissions = emissions.squeeze(0)         # Remove batch dim: [seq, num_classes]

# Decode
decoder = GreedyCTCDecoder(vocab)
decoded = decoder(emissions)

print("Decoded output:", decoded)


Decoded output: ['4', 'Sub', 'n', 'Right', '-', 'Right', '4', 'Right', 'x', 'NoRel', '-', 'Right', '2', 'Right', 'n', 'Right', '-', 'Right', '1', 'Right', 'Right', '=', 'Right', '2', 'Right', 'n', 'Right', '-', 'Right', '3']


In [ ]:
# task 10

In [67]:
def edit_distance(pred_seq: list, label_seq: list):
    m, n = len(pred_seq), len(label_seq)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if pred_seq[i - 1] == label_seq[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(
                    dp[i - 1][j],    # delete
                    dp[i][j - 1],    # insert
                    dp[i - 1][j - 1] # substitute
                )
    return dp[m][n]


In [68]:
assert (
    edit_distance(
        [
            "\\phi",
            "Right",
            "(",
            "Right",
            "0",
            "Right",
            "(",
            "Right",
            "n",
            "Right",
            ")",
            "Right",
            ")",
        ],
        [
            "\\phi",
            "Right",
            "(",
            "Right",
            "\\phi",
            "Right",
            "(",
            "Right",
            "n",
            "Right",
            ")",
            "Right",
            ")",
        ],
    )
    == 1
)

In [73]:
cd /content


/content
